In [1]:
# pip3 install cloudscraper beautifulsoup4
from bs4 import BeautifulSoup
import cloudscraper
import pandas as pd
import time
import re
from deep_translator import GoogleTranslator
from unidecode import unidecode
from urllib.parse import urljoin
import os
import requests


In [2]:
def get_url(soup):
    canonical_tag = soup.find('link', rel='canonical')
    # Lấy giá trị của thuộc tính href
    if canonical_tag and 'href' in canonical_tag.attrs:
        canonical_url = canonical_tag['href']
        return canonical_url
    return None

In [3]:
def get_id_company(soup):
    temp = get_url(soup)
    if temp:
        # Regex
        pattern = r"\/(\d+)(?=\.html)"
        match = re.search(pattern, temp)
        if match:
            return match.group(1)
    return None

In [4]:
def get_name_company(soup):
    data = soup.find("div", class_="box-detail")
    if data:
        ten_cong_ty = data.find('h1').text
        return ten_cong_ty
    return None

In [5]:
def get_web_company(soup):
    try:
        website_tag = ""
        # Tìm thẻ <a> chứa link website
        website_tag = soup.find("div", class_="company-subdetail-info website")
        if website_tag:
            website_tag = website_tag.find("a")
            return website_tag.get("href")
        else:
            return None
    except Exception as e:
        return f"Lỗi: {str(e)}"

In [6]:
def save_logo(soup):
    try:
        # Tìm tag chứa ảnh logo
        img_tag = soup.find("div", class_="company-image-logo").find("img")
        if not img_tag:
            return None
        
        # Lấy link logo
        logo_url = img_tag.get("src")
        output_folder="logos"
        logo_url = urljoin('https://cdn-new.topcv.vn', logo_url)  # Hoàn thiện URL
        # print(logo_url)
        # Lấy id công ty
        company_id = get_id_company(soup)  # Hàm này do bạn cung cấp
        
        # Tạo tên file và thư mục lưu
        filename = f"logo_{company_id}.jpg"
        os.makedirs(output_folder, exist_ok=True)
        file_path = os.path.join(output_folder, filename)
        
        # Tải ảnh từ link
        response = requests.get(logo_url, stream=True)
        with open(file_path, "wb") as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        return logo_url
    
    except Exception as e:
        return f"Lỗi: {str(e)}"

In [7]:
def get_company_size(soup):
    try:     
        # Lọc tất cả các thẻ có class "company-subdetail-info"
        size_tags = soup.find_all("div", class_="company-subdetail-info")
        for tag in size_tags:
            if "nhân viên" in (tag.text).lower():
                return tag.find("span", class_="company-subdetail-info-text").text.strip()
        return None
    except Exception as e:
        return f"Lỗi: {str(e)}"


In [8]:
def get_company_intro(soup):
    try:
        # Tìm thẻ chứa nội dung chính của công ty
        content_div = soup.find("div", class_="content")
        
        # Kiểm tra nếu thẻ <div> tồn tại
        if content_div:
            # Lấy tất cả các thẻ <p> bên trong
            paragraphs = content_div.find_all("p")
            return "\n".join(p.text.strip() for p in paragraphs)
        
        return None
    
    except Exception as e:
        return f"Lỗi: {str(e)}"

In [9]:
def get_company_address(soup):
    try:
        # Tìm tất cả các mục có class="item"
        items = soup.find_all("div", class_="item")
        
        # Duyệt qua từng mục để tìm "Địa chỉ công ty"
        for item in items:
            # Tìm phần tử có class="box-caption" và text là "Địa chỉ công ty"
            caption = item.find("div", class_="box-caption")
            if caption and "địa chỉ công ty" in (caption.get_text(strip=True)).lower():
                # Nếu tìm thấy, lấy nội dung từ class="desc"
                address = item.find("div", class_="desc")
                if address:
                    return address.get_text(strip=True)  # Trả về địa chỉ sạch
                
        return None
    
    except Exception as e:
        return f"Lỗi: {str(e)}"

In [10]:
import json

# Đường dẫn tới file JSON
file_path = 'data_26_11_sample.json'

# Đọc file JSON
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Hiển thị dữ liệu đã đọc
# print(data)


In [11]:
UrlCongTy = []
for temp in data:
    UrlCongTy.append(temp.get('UrlCongTy'))

In [ ]:
import pandas as pd
import random
import requests
# Đọc file Excel
output = []
for temp in UrlCongTy:
    while True:
        try:
            scraper = cloudscraper.create_scraper(
                        browser={
                            "browser": "chrome",
                            "platform": "windows",
                        },
                    )
            # specify the target URL
            url = temp
            response = scraper.get(url)
            print(response.status_code)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                if get_id_company(soup):
                    data = {
                        'Url': str(get_url(soup)),
                        'ID': int(get_id_company(soup)),
                        'Logo': str(save_logo(soup)),
                        'Name': str(get_name_company(soup)),
                        'CompanyWebsite': str(get_web_company(soup)),
                        'CompanySize': str(get_company_size(soup)),
                        'CompanyIntroduction': str(get_company_intro(soup)),
                        'CompanyAddress': str(get_company_address(soup))
                    }
                    output.append(data)

                    print(response.status_code, "Cập nhật thông tin thành công.", get_id_company(soup), len(output))
            if response.status_code == 429:
                # print(f"ID {ids}: Too many requests, retrying...")
                time.sleep(random.randint(1, 5))  # Chờ một khoảng thời gian trước khi thử lại
            else:
                break  # Thoát khỏi vòng lặp nếu không phải 429
        except (requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
            print(f"ID {get_id_company(soup)}: Lỗi kết nối {e}. Đang thử lại...")
            time.sleep(random.randint(1, 5))  # Chờ thêm trước khi thử lại

In [13]:
import numpy as np
# Hàm để xử lý kiểu dữ liệu không serializable
def default_converter(o):
    if isinstance(o, (np.integer, np.floating)):  # Xử lý kiểu số NumPy
        return int(o)
    elif isinstance(o, (np.ndarray,)):           # Xử lý mảng NumPy (nếu có)
        return o.tolist()
    return str(o)  # Xử lý các kiểu dữ liệu khác

In [14]:
# Lưu vào tệp JSON
with open("data_26_11_sample_cong_ty.json", "w", encoding="utf-8") as json_file:
    json.dump(output, json_file, ensure_ascii=False, default=default_converter)